In [ ]:
import sys
sys.path.insert(1, '../scr')
from PhraseDictionary import PhraseDictionary
from toolbox import Configurator, AnkiCommunicator, AnkiCardWriter
from DictionaryReader import CollinsReader, PhraseReader
from ExerciseWriter import ExerciseWriter
from Exercise import ExerciseFactory, ExerciseGatherer, ExampleSentences
import pyperclip
from pathlib import Path
import shutil
import json


def copy_folder(source_folder, destination_folder) -> None:
    destination_path = Path(destination_folder)
    source_path = Path(source_folder)
    if not destination_path.exists():
        shutil.copytree(source_path, destination_path)

### Initial set-up

In [ ]:
# User inputs
date = 20240512


# Make a new folder
template_folder = '../scr/resources/Templates'
export_folder = f'./Exports/{date}'
copy_folder(template_folder, export_folder)

# Paths of the templates
def_template_path = '/Definition/template.tex'
def_completion_template_path = '/Definition_Completion/template.tex'
reading_template_path = '/Reading/template.tex'
gap_filling_template_path = '/Fill-in-the-gap/template.tex'
paraphrase_template_path = '/Paraphrase/template.tex'
dialogue_template_path = '/Dialogue/template.tex'

# Paths of the output folder
def_output_folder = export_folder + '/Definition'
def_completion_output_folder = export_folder + '/Definition_completion'
reading_output_folder = export_folder + '/Reading'
gap_filling_output_folder = export_folder + '/Fill-in-the-gap'
paraphrase_output_folder = export_folder + '/Paraphrase'
dialogue_output_folder = export_folder + '/Dialogue'

# Creation of writers
factory = ExerciseFactory()
def_gatherer = ExerciseGatherer()
def_writer = ExerciseWriter(gatherer=def_gatherer, template_path=def_template_path, output_folder=def_output_folder)
def_completion_gatherer = ExerciseGatherer()
def_completion_writer = ExerciseWriter(gatherer=def_completion_gatherer, template_path=def_completion_template_path, output_folder=def_completion_output_folder)
reading_gatherer = ExerciseGatherer()
reading_exercise_writer = ExerciseWriter(gatherer=reading_gatherer, template_path=reading_template_path, output_folder=reading_output_folder)
gap_filling_gatherer = ExerciseGatherer()
gap_filling_exercise_writer = ExerciseWriter(gatherer=gap_filling_gatherer, template_path=gap_filling_template_path, output_folder=gap_filling_output_folder)
paraphrase_gatherer = ExerciseGatherer()
paraphrase_exercise_writer = ExerciseWriter(gatherer=paraphrase_gatherer, template_path=paraphrase_template_path, output_folder=paraphrase_output_folder)
dialogue_gatherer = ExerciseGatherer()
dialogue_exercise_writer = ExerciseWriter(gatherer=dialogue_gatherer, template_path=dialogue_template_path, output_folder=dialogue_output_folder)

In [ ]:
# category = 'Vocab Workout'
# category = 'Vocab Explorer'
category = 'Everyday English'
num_of_words_to_learn = 20


configurator = Configurator(json_path=f'../scr/config/{category}/config.json')

tmp_dict = dict()
with open('tmp.json', 'w') as file:
    json.dump(tmp_dict, file)

# Get the words to learn and review for tomorrow
tomorrow_new = configurator.get_n_words_to_learn(num_of_words_to_learn)
tmp_dict['tomorrow new'] = tomorrow_new

anki = AnkiCommunicator()
tomorrow_review = anki.get_words_for_tomorrow(deck_name=category, item='Front')
tmp_dict['tomorrow review'] = tomorrow_review

tomorrow_all = tomorrow_new + tomorrow_review
print(f'''
Number of all words: {len(tomorrow_all)};
Number of new words: {len(tomorrow_new)};
Number of old words: {len(tomorrow_review)}.
''')
with open('tmp.json', 'w') as file:
    json.dump(tmp_dict, file, indent=4)

### Definition

#### Set the word list and the set index here and get the prompt to create definitions

In [ ]:
def_start_idx = 31
def_end_idx = 40
def_set_index = 4

def_word_list = tomorrow_new[def_start_idx-1:def_end_idx]
definitions_set = factory.create_exercise(exercise_type='Definition', word_list=def_word_list)
definitions_set.import_definition_from_dictionary()
definitions_set.finish_import()
def_gatherer.import_exercise(definitions_set)
def_writer.render_template(set_index=def_set_index)

### Definition and Example Completion

#### Enter the index

In [ ]:
def_completion_start_idx = 1
def_completion_end_idx = 11
def_completion_set_index = 5

# def_completion_word_list = tomorrow_new[def_completion_start_idx-1:def_completion_end_idx]
def_completion_word_list = tomorrow_review[def_completion_start_idx-1:def_completion_end_idx]


definition_completion_exercise = factory.create_exercise(exercise_type='Definition and Example Completion', 
                                                         word_list=def_completion_word_list)
definition_completion_exercise.generate_exercise()
definition_completion_exercise.finish_import()
def_completion_gatherer.import_exercise(definition_completion_exercise)
def_completion_writer.render_template(def_completion_set_index)

### Reading

#### Enter the set index and the word list here

In [ ]:
reading_start_idx = 1
reading_end_idx = 15
reading_set_index = 1
reading_word_list = tomorrow_review[reading_start_idx-1:reading_end_idx]
reading_exercise = factory.create_exercise(exercise_type='Reading', word_list=reading_word_list)

#### Paste the initial passage here

In [ ]:
initial_passage = pyperclip.paste()
reading_exercise.create_prompt(passage=initial_passage)
reading_exercise.get_prompt()

#### Paste the final passage here

In [ ]:
final_passage = pyperclip.paste()

reading_exercise.import_passage(text=final_passage)

#### Paste the multiple choice questions here

In [ ]:
multiple_choice = pyperclip.paste()
reading_exercise.import_exercise(text=multiple_choice)

##### Paste the solution here

In [ ]:
reading_solution = pyperclip.paste()
reading_exercise.import_solution(text=reading_solution)
reading_exercise.finish_import()
reading_gatherer.import_exercise(exercise=reading_exercise)
reading_exercise_writer.render_template(set_index=reading_set_index)

### Vocabulary exercises

#### Set the word list here and get the prompt to create example sentences

In [ ]:
vocab_start_idx = 1
vocab_end_idx = 11
repetition = 'two'

# vocab_word_list = tomorrow_new[vocab_start_idx-1:vocab_end_idx]
vocab_word_list = tomorrow_review[vocab_start_idx-1:vocab_end_idx]

example_sentences = ExampleSentences(vocab_word_list)
example_sentences.create_prompt(number_of_sentences=repetition)
example_sentences.get_prompt()

#### Paste the example sentences here

In [ ]:
sentences = pyperclip.paste()
example_sentences.import_sentences(text=sentences)
example_sentences.finish_import()

#### Fill in the gap

##### Enter the set index here

In [ ]:
gap_filling_set_index = 5

gap_filling_exercise = factory.create_exercise(exercise_type='Fill in the gap', word_list=example_sentences.word_list, example_sentences=example_sentences)
gap_filling_exercise.finish_import()
gap_filling_gatherer.import_exercise(exercise=gap_filling_exercise)
gap_filling_exercise_writer.render_template(set_index=gap_filling_set_index)

#### Paraphrase

In [ ]:
pyperclip.copy(example_sentences.get_second_prompt())

##### Paste the json string here

In [ ]:
analyzed_sentences = pyperclip.paste()
example_sentences.import_sentence_analysis(text=analyzed_sentences)

##### Enter the set index here

In [ ]:
paraphrase_set_index = 1

paraphrase_exercise = factory.create_exercise(exercise_type='Paraphrase', word_list=example_sentences.word_list, example_sentences=example_sentences)
paraphrase_exercise.finish_import()
paraphrase_gatherer.import_exercise(exercise=paraphrase_exercise)
paraphrase_exercise_writer.render_template(set_index=paraphrase_set_index)

### Dialogue

#### Enter the set index here

In [ ]:
dialogue_start_idx = 11
dialogue_end_idx = 20
dialogue_set_index = 2

dialogue_word_list = tomorrow_new[dialogue_start_idx-1:dialogue_end_idx]
# dialogue_word_list = tomorrow_review[dialogue_start_idx-1:dialogue_end_idx]
dialogue_exercise = factory.create_exercise(exercise_type='Dialogue', word_list=dialogue_word_list)
dialogue_exercise.get_prompt()

In [ ]:
dialogue_dict_text = pyperclip.paste()
dialogue_dict = json.loads(dialogue_dict_text)
dialogue_exercise.generate_exercise(dialogue_dict=dialogue_dict)
dialogue_exercise.finish_import()
dialogue_gatherer.import_exercise(exercise=dialogue_exercise)
dialogue_exercise_writer.render_template(set_index=dialogue_set_index)

## Select the dictionary reader and write Anki Cards

In [ ]:
reader = PhraseReader(word_list=tomorrow_new)

entry_list = reader.get_word_entry_list()
writer = AnkiCardWriter(entry_list)
Path(f'Exports/{date}_{category}').mkdir(parents=True, exist_ok=True)
writer.write_cards(f'Exports/{date}_{category}/Anki Cards.csv')

## Finish

In [ ]:
configurator.study_n_words(num_of_words_to_learn)

## Add Phrases

In [ ]:
notes_paths = 'test.json'

phrase_dictionary = PhraseDictionary()
phrase_dictionary.import_initial_JSON(notes_paths)
phrase_dictionary.get_prompt()

In [ ]:
refined_dict_text = pyperclip.paste()
refined_dict = json.loads(refined_dict_text)
phrase_dictionary.import_new_entries(refined_dict)